In [15]:
from datasets import load_dataset
from transformers import BertTokenizer

# Charger le dataset Swissprot-families
dataset = load_dataset("khairi/Swissprot-families")

# Utiliser le tokenizer BERT pré-entrainé
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["Sequence"], padding="max_length", truncation=True)

# Tokenizer les séquences
tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [16]:
# Extraire les labels uniques de tous les splits
all_labels = set(tokenized_datasets["train"]["Organism"])
all_labels.update(tokenized_datasets["validation"]["Organism"])
all_labels.update(tokenized_datasets["test"]["Organism"])

# Convertir en liste triée pour avoir un ordre consistant
labels = sorted(list(all_labels))
num_labels = len(labels)

# Encoder les labels
def encode_labels(examples):
    examples["label"] = [labels.index(label) for label in examples["Organism"]]
    return examples

# Ajouter les labels encodés au dataset tokenisé
tokenized_datasets = tokenized_datasets.map(encode_labels, batched=True)


Map: 100%|██████████| 5000/5000 [00:00<00:00, 10114.54 examples/s]


In [17]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Charger le modèle BERT pré-entrainé avec un classificateur de séquence
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Configurer les arguments de l'entrainement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Entrainement du modèle
trainer.train()


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

KeyboardInterrupt: 

In [ ]:
# Évaluation sur le dataset de test
results = trainer.evaluate(tokenized_datasets["test"])
print(results)
